# 스코어 계산

## 목적
- 중증도 스코어와 경제부담 스코어를 계산
- 계산된 스코어를 CSV 파일로 저장
- 결과_분석.ipynb에서 사용할 데이터 준비

In [1]:
import os
os.chdir(r"c:\Users\user\OneDrive\바탕 화면\2025-2\DSL\EDA 프로젝트\dataset\전처리")
print(f"현재 작업 디렉토리: {os.getcwd()}")

현재 작업 디렉토리: c:\Users\user\OneDrive\바탕 화면\2025-2\DSL\EDA 프로젝트\dataset\전처리


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

## 건강보험/의료급여 본인부담률, 본인부담률 증가율 계산

In [9]:
# 데이터 불러오기
df = pd.read_csv("merged_preprocessed_with_22.csv", encoding='cp949')

# 필수 열만 선택
df = df[[
    '연령', '298 질병분류', '진료형태',
    '진료실인원수', '진료실인원수(급여)', 
    '진료비', '급여비',
    '총진료비(급여)', '기금부담금(급여)'
]].copy()

# 숫자형 변환
for col in ['진료실인원수', '진료실인원수(급여)', '진료비', '급여비', '총진료비(급여)', '기금부담금(급여)']:
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

# 건강보험 본인부담금 계산 및 삽입
df['건강보험_본인부담금'] = df['진료비'] - df['급여비']

# 본인부담률 계산
df['건강보험_본인부담률'] = (df['진료비'] - df['급여비']) / df['진료비']
df['의료급여_본인부담률'] = (df['총진료비(급여)'] - df['기금부담금(급여)']) / df['총진료비(급여)']

# 추가 컬럼들 계산 
df['본인부담금'] = df['진료비'] - df['급여비']
df['본인부담금(급여)'] = df['총진료비(급여)'] - df['기금부담금(급여)']
df['1인당_본인부담금'] = df['본인부담금'] / df['진료실인원수']
df['1인당_본인부담금(급여)'] = df['본인부담금(급여)'] / df['진료실인원수(급여)']

# '입원' 항목만 추출
df_inpatient = df[df['진료형태'] == '입원'].copy()

def parse_age(age_str):
    if '세' in age_str:
        return 0
    elif '-' in age_str:
        return int(age_str.split('-')[0])
    else:
        return 999

df_inpatient['연령_정렬값'] = df_inpatient['연령'].apply(parse_age)

# 연령별 본인부담률 테이블 생성 (입원 데이터 기준)
pivot_health = df_inpatient.pivot_table(index=['298 질병분류'], columns='연령_정렬값', values='건강보험_본인부담률')
pivot_medical = df_inpatient.pivot_table(index=['298 질병분류'], columns='연령_정렬값', values='의료급여_본인부담률')

# 각 연령대별 기울기(증가율 또는 변화량) 계산
slope_health = pivot_health.diff(axis=1)
slope_medical = pivot_medical.diff(axis=1)

# 컬럼 이름 정렬된 연령대 이름으로 다시 매핑 (선택 사항)
age_sorted = df_inpatient[['연령', '연령_정렬값']].drop_duplicates().sort_values('연령_정렬값')
age_colnames = age_sorted.set_index('연령_정렬값')['연령'].to_dict()
pivot_health.columns = [age_colnames.get(col, str(col)) for col in pivot_health.columns]
pivot_medical.columns = [age_colnames.get(col, str(col)) for col in pivot_medical.columns]
slope_health.columns = ['기울기_' + age_colnames.get(col, str(col)) for col in slope_health.columns]
slope_medical.columns = ['기울기_' + age_colnames.get(col, str(col)) for col in slope_medical.columns]

# 결과 저장
pivot_health.to_csv("연령대별_건강보험_본인부담률.csv")
pivot_medical.to_csv("연령대별_의료급여_본인부담률.csv")
slope_health.to_csv("연령대별_건강보험_기울기_new.csv")
slope_medical.to_csv("연령대별_의료급여_기울기_new.csv")

merged_df = df.copy()  

print("모든 파일 저장 완료")

모든 파일 저장 완료


## 중증도 스코어와 경제부담 스코어 계산

In [ ]:
# 경제적 부담 관련 데이터 불러오기 (치명률 제외)
rate_health    = pd.read_csv("연령대별_건강보험_본인부담률.csv")
grad_health    = pd.read_csv("연령대별_건강보험_기울기_new.csv")
rate_medical   = pd.read_csv("연령대별_의료급여_본인부담률.csv")

# '입원' 필터링 → 1인당 본인부담금 계산
merged_df_inpatient = merged_df[merged_df['진료형태'] == '입원']
burden_pp = (
    merged_df_inpatient
    .groupby(['298 질병분류','연령'])[['본인부담금','진료실인원수']]
    .sum()
    .assign(본인부담금=lambda d: d['본인부담금'] / d['진료실인원수'])
    .reset_index()
)
pivot_burden = burden_pp.pivot(index='298 질병분류', columns='연령', values='본인부담금')

# 연령대 컬럼 목록
age_cols  = [c for c in rate_health.columns if c not in ['298 질병분류','계']]
grad_cols = ['기울기_' + a for a in age_cols]

# 질병별 경제적 부담 지표 계산
df_burden = pd.DataFrame({'298 질병분류': rate_health['298 질병분류']})
df_burden['avg_본인부담률']      = rate_health[age_cols].mean(axis=1)
df_burden['avg_기울기']          = grad_health[grad_cols].mean(axis=1)
rate_diff = rate_health.copy()
rate_diff[age_cols] = rate_health[age_cols] - rate_medical[age_cols]
df_burden['avg_보장_격차']       = rate_diff[age_cols].mean(axis=1)
avg_burden = pivot_burden[age_cols].mean(axis=1)
df_burden['avg_1인당_본인부담금'] = df_burden['298 질병분류'].map(avg_burden)

print("경제적 부담 관련 데이터:")
print(df_burden.head())

# 경제적 부담 변수들 결측치 대체 및 정규화
features_burden = [
    'avg_본인부담률',
    'avg_기울기', 
    'avg_보장_격차',
    'avg_1인당_본인부담금'
]

# 결측치 대체 전략 적용
df_burden_imputed = df_burden.copy()

# 결측치 대체 적용
# 기울기: 중앙값 대체
median_slope = df_burden_imputed['avg_기울기'].median()
df_burden_imputed['avg_기울기'] = df_burden_imputed['avg_기울기'].fillna(median_slope)

# 보장격차: 중앙값 대체
median_gap = df_burden_imputed['avg_보장_격차'].median()
df_burden_imputed['avg_보장_격차'] = df_burden_imputed['avg_보장_격차'].fillna(median_gap)

# 본인부담률: 평균값 대체
mean_rate = df_burden_imputed['avg_본인부담률'].mean()
df_burden_imputed['avg_본인부담률'] = df_burden_imputed['avg_본인부담률'].fillna(mean_rate)

# 본인부담금: 평균값 대체
mean_burden = df_burden_imputed['avg_1인당_본인부담금'].mean()
df_burden_imputed['avg_1인당_본인부담금'] = df_burden_imputed['avg_1인당_본인부담금'].fillna(mean_burden)

# '계', '기타' 등 문자열 데이터 제거
df_burden_clean = df_burden_imputed[
    ~df_burden_imputed['298 질병분류'].isin(['계', '기타'])
].copy()

# 정규화
scaler_burden = MinMaxScaler()
df_burden_clean[features_burden] = scaler_burden.fit_transform(df_burden_clean[features_burden])

# 경제적 부담 스코어 계산 (PCA)
pca_burden = PCA(n_components=1)
df_burden_clean['경제부담_스코어'] = pca_burden.fit_transform(df_burden_clean[features_burden])
df_burden_clean['경제부담_스코어'] = MinMaxScaler().fit_transform(df_burden_clean[['경제부담_스코어']])

print(f"\n■ 경제적 부담 스코어 PCA 가중치:")
burden_loadings = np.abs(pca_burden.components_[0])
burden_weights = burden_loadings / burden_loadings.sum()
for feature, weight in zip(features_burden, burden_weights):
    print(f"  - {feature}: {weight:.4f}")

# 치명률 데이터 불러오기 및 중증도 스코어 계산
mortality = pd.read_csv("치명률.csv", encoding='cp949')
mort_2023 = (
    mortality[mortality['기준연도'] == 2023]
    .rename(columns={'치명률 (실인원수 기준)': '치명률'})
    [['298 질병분류', '치명률']]
)

# 데이터 타입 통일 및 안전한 변환
print("\n■ 데이터 타입 변환 중...")
print(f"  경제부담 데이터: {len(df_burden_clean)}개 질병")
print(f"  치명률 데이터: {len(mort_2023)}개 질병")

# 안전한 숫자 변환
df_burden_clean['298 질병분류'] = pd.to_numeric(df_burden_clean['298 질병분류'], errors='coerce')
mort_2023['298 질병분류'] = pd.to_numeric(mort_2023['298 질병분류'], errors='coerce')

# NaN 제거 후 정수 변환
df_burden_clean = df_burden_clean.dropna(subset=['298 질병분류'])
mort_2023 = mort_2023.dropna(subset=['298 질병분류'])

df_burden_clean['298 질병분류'] = df_burden_clean['298 질병분류'].astype(int)
mort_2023['298 질병분류'] = mort_2023['298 질병분류'].astype(int)

print(f"  변환 후 경제부담 데이터: {len(df_burden_clean)}개 질병")
print(f"  변환 후 치명률 데이터: {len(mort_2023)}개 질병")

# 치명률 병합
df_final = df_burden_clean.merge(mort_2023, on='298 질병분류', how='left')
df_final['치명률'] = df_final['치명률'].fillna(0)  # 결측값은 0으로 처리

# 중증도 스코어 계산
scaler_severity = MinMaxScaler()
df_final['중증도_스코어'] = scaler_severity.fit_transform(df_final[['치명률']])

# 결과 정리 및 저장
df_result = df_final[[
    '298 질병분류', '중증도_스코어', '경제부담_스코어'
]].copy()

# 컬럼명 변경
df_result = df_result.rename(columns={'298 질병분류': '질병분류'})

# 각 스코어별로 정렬하여 저장
severity_result = df_result.sort_values('중증도_스코어', ascending=False)
burden_result = df_result.sort_values('경제부담_스코어', ascending=False)

severity_result.to_csv("질병별_중증도_스코어.csv", index=False)
burden_result.to_csv("질병별_경제부담_스코어.csv", index=False)

# 결과_분석.ipynb에서 사용할 통합 스코어 데이터도 저장
df_result.to_csv("통합_스코어_데이터.csv", index=False)

print("완료: 분리된 스코어 파일 저장됨")
print("  - '질병별_중증도_스코어.csv' 저장됨")
print("  - '질병별_경제부담_스코어.csv' 저장됨")
print("  - '통합_스코어_데이터.csv' 저장됨")

print("\n상위 10개 중증도 스코어:")
print(severity_result.head(10))

print("\n상위 10개 경제부담 스코어:")
print(burden_result.head(10))

경제적 부담 관련 데이터:
  298 질병분류  avg_본인부담률   avg_기울기  avg_보장_격차  avg_1인당_본인부담금
0        1   0.209984  0.016090   0.191502     254.750000
1       10   0.263245       NaN   0.196514    1637.000000
2      100   0.147406  0.000288   0.091490    1108.478969
3      101   0.229839 -0.031035   0.183801     254.378571
4      102   0.240979  0.003613   0.196293     587.276018

■ 경제적 부담 스코어 PCA 가중치:
  - avg_본인부담률: 0.4804
  - avg_기울기: 0.1020
  - avg_보장_격차: 0.3590
  - avg_1인당_본인부담금: 0.0585

■ 데이터 타입 변환 중...
  경제부담 데이터: 286개 질병
  치명률 데이터: 297개 질병
  변환 후 경제부담 데이터: 286개 질병
  변환 후 치명률 데이터: 297개 질병
완료: 분리된 스코어 파일 저장됨
  - '질병별_중증도_스코어.csv' 저장됨
  - '질병별_경제부담_스코어.csv' 저장됨
  - '통합_스코어_데이터.csv' 저장됨

상위 10개 중증도 스코어:
     질병분류   중증도_스코어  경제부담_스코어
76     17  1.000000  0.379789
247    64  0.688965  0.318222
248    65  0.466769  0.298565
246    63  0.360766  0.300013
267    82  0.356076  0.244435
250    67  0.354616  0.286255
241    59  0.345667  0.286260
163   248  0.321663  0.107691
185   269  0.283316  0.663418
15  